# **IMPORTS/COMFIGIRAÇÕES**

In [2]:
# Bibliotecas para manipulação de dados
import math
import numpy as np  # Manipulação de matrizes
import pandas as pd  # Manipulação de dados tabulares
import datetime  # Manipulação de data e hora
# Bibliotecs de visualização gráfica
import plotly.express as px
# Biblioiteca de deploy
import streamlit as st
# Bibliotecas próprias
from biblioteca.data_extraction import *  # Biblioteca própria para extração dos dados
from biblioteca.data_transform import *  # Biblioteca própria para transformação dos dados
# Biblioteca de Ml
from pycaret.time_series import *
# Biblioteca de filtro de notificações
import warnings



warnings.filterwarnings(action='ignore')
pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None
pd.options.display.float_format = lambda x: f'{x:,.2f}'
px.defaults.template = 'plotly_dark'

# **LOAD**

In [3]:
def load(path):
    df = pd.read_csv(path, parse_dates=['COMPETÊNCIA'], index_col='COMPETÊNCIA')
    df_e = df[df.ESFERA == 'Estadual']
    df_m = df[df.ESFERA == 'Municipal']
    return df_e, df_m

df_e, df_m = load('./data.csv')
display(df_e.sample(5))

,ESFERA,UF,REPASSE,TOTAL AJUSTE,TOTAL LIQUIDO,CATEGORIA
COMPETÊNCIA,,,,,,
2019-06-01,Estadual,MS,COUN_VAAF,0.00,0.00,Complementação VAAF
2016-04-01,Estadual,RO,FPE,0.00,"19,020,039.17",FPE
2021-11-01,Estadual,PA,AFE_EC123,0.00,0.00,Outros
2021-11-01,Estadual,BA,AFE_EC123,0.00,0.00,Outros
2021-02-01,Estadual,AP,IPVA,0.00,"869,048.86",IPVA


In [4]:
y = df_e.pivot_table(values='TOTAL LIQUIDO', index=df_e.index, columns='UF', aggfunc='sum')
# y['TOTAL'] = y.sum(axis='columns')
display(y.head())

UF,AC,AL,AM,AP,BA,CE,DF,ES,GO,MA,MG,MS,MT,PA,PB,PE,PI,PR,RJ,RN,RO,RR,RS,SC,SE,SP,TO
COMPETÊNCIA,,,,,,,,,,,,,,,,,,,,,,,,,,,
2007-01-01,"14,412,989.93","10,256,521.49","27,943,538.46","16,311,777.23","36,042,237.72","14,047,735.13","4,119,975.11","27,378,954.14","52,854,486.08","14,365,535.24","171,622,071.48","20,957,464.96","29,914,439.53","19,519,590.38","21,924,173.82","42,204,491.65","11,191,635.49","90,383,629.57","64,999,520.74","22,701,880.35","17,562,090.39","11,999,428.43","123,815,064.83","58,206,262.12","16,421,156.48","525,149,806.34","16,331,573.81"
2007-02-01,"14,895,257.97","12,372,527.84","26,074,852.23","14,264,167.26","37,708,281.85","12,246,894.26","4,405,280.19","25,905,547.03","50,274,278.17","15,315,952.18","167,815,142.32","26,552,684.41","30,148,986.12","25,482,789.10","23,527,859.40","40,442,860.29","11,030,537.14","82,507,363.97","57,226,412.94","19,036,464.72","15,352,611.26","14,331,052.38","83,224,524.59","57,328,063.26","14,789,985.46","434,965,363.38","20,864,891.72"
2007-03-01,"14,225,224.33","16,250,129.21","27,911,816.02","15,180,756.22","61,975,698.68","28,426,188.24","4,061,415.32","27,856,321.06","45,710,034.82","28,928,186.30","186,752,113.83","26,855,001.99","33,733,511.47","38,411,227.09","23,378,471.74","46,362,054.88","16,608,286.96","95,838,035.11","74,956,450.10","22,295,037.15","18,997,014.14","12,917,484.11","113,189,147.54","62,453,074.28","15,703,289.80","508,425,560.22","19,749,986.59"
2007-04-01,"20,285,084.39","21,717,712.66","29,786,641.18","25,168,447.00","99,615,476.46","45,660,527.87","6,231,685.34","30,356,776.68","61,246,992.20","38,933,913.36","302,345,363.00","39,414,890.22","41,955,776.15","53,063,167.90","32,123,412.83","69,724,630.53","26,536,990.40","136,847,785.67","107,367,593.10","33,171,021.46","25,535,205.37","17,568,022.79","163,349,996.28","89,411,902.60","22,967,045.93","766,287,139.15","30,075,396.07"
2007-05-01,"17,085,455.69","19,906,997.44","45,030,962.51","19,161,892.14","76,415,254.62","31,338,354.51","5,209,085.47","35,417,356.79","48,829,044.86","37,097,936.09","211,118,055.86","32,006,864.98","36,333,072.55","43,936,551.48","31,375,641.96","58,046,093.80","19,670,804.95","108,451,531.06","83,545,085.17","28,737,192.39","34,476,939.13","17,152,502.52","123,376,673.63","71,886,420.93","19,790,261.17","614,216,307.11","25,433,631.94"


In [24]:
y.shape

(204, 28)

# **ANÁLISE PREDITIVA**

In [5]:
Y = y['AP'][0:203]
Y.tail()

COMPETÊNCIA
2023-07-01   81,646,350.50
2023-08-01   68,821,655.09
2023-09-01   62,534,438.50
2023-10-01   64,892,671.59
2023-11-01   87,170,101.56
Name: AP, dtype: float64

In [7]:
fh = math.ceil((datetime.datetime(year=datetime.datetime.today().year, month=12, day=31)-datetime.datetime.today()).days/30)

s = setup(data=Y, fold=5, fh=fh, session_id=1)

,Description,Value
0,session_id,1
1,Target,AP
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(203, 1)"
5,Transformed data shape,"(203, 1)"
6,Transformed train set shape,"(191, 1)"
7,Transformed test set shape,"(12, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [8]:
best = compare_models()

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
ets,ETS,1.5862,1.3704,7233025.7012,9178516.7647,0.1221,0.1286,-0.0619,0.0260
theta,Theta Forecaster,1.6228,1.3993,7419542.3481,9396442.9614,0.1247,0.1317,-0.0873,0.0120
exp_smooth,Exponential Smoothing,1.6290,1.3893,7455717.6743,9324279.4123,0.1250,0.1312,-0.0924,0.0240
auto_arima,Auto ARIMA,1.6851,1.4035,7718131.6528,9410694.3913,0.1281,0.1359,-0.1112,11.8500
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.7160,1.4823,7974446.5027,10017226.1646,0.1353,0.1374,-0.2263,0.0740
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,1.7351,1.4273,8088395.6462,9709894.3158,0.1337,0.1390,-0.1085,0.1160
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,1.7390,1.5065,8102486.5373,10207149.9184,0.1354,0.1393,-0.2292,0.1160
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,1.7565,1.4490,8197077.9146,9863439.0770,0.1356,0.1411,-0.1424,0.0620
stlf,STLF,1.7718,1.4657,8057374.3299,9781420.7773,0.1399,0.1440,-0.2150,0.0160
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,1.7798,1.4649,8316269.7008,9960047.7021,0.1387,0.1432,-0.1648,0.1440


In [9]:
plot_model(best, plot='forecast', data_kwargs={'fh': 24})

x - Qual a previsão mensal dos repasses até o exercício seguinte?

In [11]:

pred = predict_model(estimator=best, fh=fh)
pred.index = pd.to_datetime([x.to_timestamp() for x in list(pred.index)])
pred = pred.resample(rule='M').sum()

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,Theta Forecaster,1.0110,0.9463,542292246.9656,666566305.5060,0.0638,0.0667,0.2401


In [12]:
df_temp = pd.merge(right=preditive_df, left=pred, left_index=True, right_index=True, how='outer')
df_temp.loc[(df_temp.index <= '2023-11-1'), 'y_pred'] = np.nan
df_temp.columns = ['Previsão', 'Histórico']


fig = px.line(data_frame=df_temp, y=['Previsão', 'Histórico'], x=df_temp.index)
fig.show()

x - Qual a previsão mensal dos repasses até o exercício seguinte para cada estado?